Produce Light Curve for every star

In [1]:
from common import * 

/Users/thomas/.brite


In [2]:
import matplotlib.pyplot as plt
import numpy as np
plt.rcParams.update({'font.size': 15, 'xtick.labelsize': 'small', 'ytick.labelsize': 'small','ytick.direction': 'in', 'xtick.direction': 'in',  # tells matplotlib to plot the ticks inward
                     'ytick.right': True, 'xtick.top': True, # tells matplotlib to plot the ticks also on the right and on the top
                     'xtick.minor.visible': True,'ytick.minor.visible': True, # include minor ticks as well
                     'xtick.major.width': 2, 'ytick.major.width': 2 # widht of major ticks
					 , 'ytick.minor.width': 1.25, 'xtick.minor.width': 1.25,})

In [3]:
config_dict = get_config()
default_result_entry = 'all'


In [4]:
def color(path):
    if 'BAb' in path or 'BLb' in path:
        return 'royalblue'
    elif 'BHr' in path or 'UBr' in path or 'BTr' in path:
        return 'firebrick'
    else:
        raise NameError('No Satellite Name in Path: ' + path)



def create_plot(path):
    name = path.split('/')[-1]
    data_path = os.path.join(config_dict['Decorrelation Path'],path)
    try:
        data = Data(path,None)
    except:
        return
    mosaic = """
            AABCD
            AABED
            """
    fig = plt.figure(constrained_layout=True,figsize = (15,5),dpi = 100)
    axes = fig.subplot_mosaic(mosaic)
    lk = data._lk_obj
    do_ave = True
    try:
        ave_lk = data._ave_lk_obj
    except OS_Error:
        do_ave = False
    
    
    zoom_in = lk[abs(lk.time.value - np.mean(lk.time.value)) <= 5]
    zoom_in_2 = lk[abs(lk.time.value - np.mean(lk.time.value) - 10) <= 5]
    if len(zoom_in_2.time.value) > len(zoom_in.time.value):
        zoom_in = zoom_in_2
    zoom_in_3 = lk[abs(lk.time.value - np.mean(lk.time.value) + 10) <= 5]
    if len(zoom_in_3.time.value) > len(zoom_in.time.value):
        zoom_in = zoom_in_3
    
    
    c = color(name)
    
    axes["A"].plot(lk.time.value, lk.flux.value, 'ko', ms = 0.5, alpha = 0.5)
    axes["A"].axvline(zoom_in.time.value[0], color = 'silver', lw = 1, ls = '--', zorder = 5)
    axes["A"].axvline(zoom_in.time.value[-1], color = 'silver', lw = 1, ls = '--', zorder = 5)
    axes["B"].plot(zoom_in.time.value, zoom_in.flux.value, 'ko', ms = 0.5, alpha = 0.5)
    
    if do_ave:
        ave_zoom_in = ave_lk[abs(ave_lk.time.value - zoom_in.time.value[0] - 5) <= 5]
        axes["B"].plot(ave_zoom_in.time.value, ave_zoom_in.flux.value,  color = c, ls = '', marker = 'o',  ms = 1.5, alpha = 0.75)
        axes["A"].plot(ave_lk.time.value, ave_lk.flux.value,  color = c, ls = '', marker = 'o', ms = 1.5, alpha = 0.75)
    
    pdg = data.to_periodogram(minimum_frequency = 0.1, maximum_frequency = 100)
    period = pdg.period_at_max_power.value
    period2 = 2*period
    print(period)
    

    
    
    axes["C"].plot((lk.time.value%period)/period, lk.flux.value, 'ko', ms = 0.5, alpha = 0.5)
    axes["C"].plot((lk.time.value%period)/period + 1, lk.flux.value, 'ko', ms = 0.5, alpha = 0.5)
    axes["E"].plot((lk.time.value%period2)/period2, lk.flux.value, 'ko', ms = 0.5, alpha = 0.5)
    axes["E"].plot((lk.time.value%period2)/period2 + 1, lk.flux.value, 'ko', ms = 0.5, alpha = 0.5)
    axes["D"].axvline(1/period, color = c, lw = 2, ls = '-')
    axes["D"].axvline(1/period2, color = c, lw = 2, ls = '--')
    axes["D"].plot(pdg.frequency, pdg.power, 'k-', lw = 1)
    
    
    if do_ave:
        axes["C"].plot((ave_lk.time.value%period)/period, ave_lk.flux.value, color = c, ls = '', marker = 'o', ms = 1.5, alpha = 0.75)
        axes["C"].plot((ave_lk.time.value%period)/period + 1, ave_lk.flux.value, color = c, ls = '', marker = 'o', ms = 1.5, alpha = 0.75)
        axes["E"].plot((ave_lk.time.value%period2)/period2, ave_lk.flux.value,  color = c, ls = '', marker = 'o', ms = 1.5, alpha = 0.75)
        axes["E"].plot((ave_lk.time.value%period2)/period2 + 1, ave_lk.flux.value, color = c, ls = '', marker = 'o', ms = 1.5, alpha = 0.75)
    
    axes["D"].set_xlim(0, np.max([10, 2/period]))
    axes["B"].set_yticklabels([])
    x1 = np.round(zoom_in.time.value[0] + 2, 0)
    x3 = np.round(zoom_in.time.value[0] + 8, 0)
    x2 = np.round((x1 + x3)/2, 0)
    axes["B"].set_xticks([x1, x2, x3])
    axes["C"].set_yticklabels([])
    axes["E"].set_yticklabels([])
    axes['D'].yaxis.tick_right()
    axes['D'].yaxis.set_label_position('right')
    axes['C'].xaxis.set_label_position('top') 
    axes["A"].invert_yaxis()
    axes["B"].invert_yaxis()
    axes["C"].invert_yaxis()
    axes["E"].invert_yaxis()
    axes["A"].set_xlabel("time (days)")
    axes["B"].set_xlabel("time (days)")
    axes["C"].set_xlabel("phase P={:3f}d".format(period))
    axes["E"].set_xlabel("phase P={:3f}d".format(period2))
    axes["D"].set_xlabel("frequency ($d^{-1}$)")
    axes["A"].set_ylabel("Magnitude")
    axes["D"].set_ylabel("Power")
    axes["D"].set_title("f={:3f} ".format(1/period) + '$d^{-1}$')
    ylim = axes["A"].get_ylim()
    axes["B"].set_ylim(ylim)
    axes["C"].set_ylim(ylim)
    axes["E"].set_ylim(ylim)
    fig.suptitle(name)
    plt.tight_layout()
    plt.subplots_adjust(wspace=0, hspace=0)
    plt.savefig(f"../LC_overview_plot/{name}.png", facecolor = "white")
    
    

In [8]:
num = 0
for field in [9, 10, 11, 12, 14]:
# for field in [14]:
    stars = load(field)
    num += len(stars)
    print("Numbers of stars done:", num)
    for star in stars:
        result_path = star.results[0]
        for dataset in star.get_all_data_sets(result_path):
            if dataset.combined:
                create_plot(dataset._path)
            
#     if result_path == default_result_entry:
#             used_path = star._path
#     else:
#         used_path = os.path.join(star._path,result_path)
#     objects = {}

#     counter = 1

#     for root,dir,files in os.walk(used_path):
#         files = [i for i in files if i.endswith('ndat')]
#         for file in files:
#             objects[counter] = (file,os.path.join(root,file))
#             counter +=1
    
#     keys = objects.keys()
#     paths = [objects[key][1] for key in keys]
    
#     print(paths)
#     for p in paths:
#         if '_ME' in p:
#             create_plot(p)
    

        
        
        
#         path =  objects[key][1]
#         data = Data(path,None)
#         print(data)
#         data.scatter()
#     print(objects.keys())

127
